In [ ]:
'''
This is a notebook to visualize a maritime traffic network
- load a network from pickle
- plot the network and add some trajectories (optional)
- plot the network graph on a canvas
'''

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from ast import literal_eval
import folium
import time
import warnings
import pickle
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

In [ ]:
# add paths for modules
sys.path.append('../models')
# import modules
import  visualize
from maritime_traffic_network import MaritimeTrafficNetwork

In [ ]:
# load network from pickle
model_date = '202204' #"['202204']"
location = 'stavanger'
DP_tol = 10
min_samples = 16
data_size = 'full'
refined = '_refined'

model_name = model_date+'_waypoints_DP'+str(DP_tol)+'_HDBSCAN'+str(min_samples)+'_'+location+'_'+data_size+'_UTM'+refined
model_path = '../../models/networks/refined_networks/' + model_name + '.obj'
fileObj = open(model_path, 'rb')
network = pickle.load(fileObj)
fileObj.close()
network.hyperparameters

In [ ]:
# plot network and basemap
network.prune_graph(5, 'refined')
map = network.map_graph(refined=True, line_weight=2, min_passages=1)  #tromso: center=[69, 19], oslo: center=[59, 10.5]

# add some trajectories
all_trajectories = network.trajectories
selection_start = 0
selection_end = 1 #len(all_trajectories)
selection_step = 1
selection = np.arange(selection_start, selection_end, selection_step)
mmsis = network.gdf.mmsi.unique()[selection]

for mmsi in mmsis:
    print(mmsi)
    trajectory = all_trajectories.get_trajectory(mmsi)
    path_df_sspd, evaluation_results_sspd = network.trajectory_to_path_sspd(trajectory, verbose=False)
    print(evaluation_results_sspd[['message', 'SSPD']])
    print('------------------')
    try:
        map = trajectory.to_line_gdf()[['geometry', 'mmsi', 'skipsgruppe']].explore(m=map, style_kwds={'weight':3, 'color':'black', 'opacity':1}, 
                                                                      name=mmsi+' trajectory')
        map = path_df_sspd.explore(m=map, name=mmsi+' edge_sequence sspd', style_kwds={'weight':3, 'color':'cyan', 'opacity':1})
    except:
        print(mmsi+': no path found')

bbox = visualize.get_bounding_box(network.gdf)
map = bbox.explore(m=map, color='red', name='bounds')
folium.LayerControl().add_to(map)
map

In [ ]:
# Save map to file
map.save('../../reports/maps/'+model_name+'.html')
print(f'Map saved under {"../../reports/maps/"+model_name+".html"}')

In [ ]:
# plot graph on a canvas
network.plot_graph_canvas(pruned=True)

In [ ]:
# Calculate average out-degree
# Calculate out-degrees of all nodes
out_degrees = network.G.out_degree()
average_out_degree = sum(out_degree for node, out_degree in out_degrees) / len(out_degrees)
print("Average Out-Degree:", average_out_degree)